In [1]:
import pandas as pd
import nltk, re, string
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Embedding, Input,LSTM,GlobalMaxPool1D, Dropout
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.preprocessing.text import one_hot
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
true_df = pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\True.csv")
fake_df=pd.read_csv("C:\\Users\\simra\\Downloads\\NLP Project\\News-_dataset\\Fake.csv")

In [3]:
#Add label Column
true_df['label'] = 0
fake_df['label'] = 1
#true_df

In [7]:
#Combined Both Files 
combined_df = pd.concat([true_df, fake_df], ignore_index=True)
#combined_df

In [9]:
combined_df=combined_df.drop(['date'],axis=1)

In [11]:
combined_df['Text'] = combined_df['title'] + " " + combined_df['text'] + " " + combined_df['subject']
combined_df=combined_df.drop(['title','text','subject'],axis=1)

In [12]:
y = combined_df['label'].values
X = combined_df.drop(['label'],axis=1)

In [15]:
#Remove Reuters word From True News
def Remove_Reuters(text):
    regex = re.search(r"(?<=\(Reuters\)\s\-\s).*",text)
    if regex:
        return regex.group(0)
    return text
X['Text'] = X['Text'].apply(Remove_Reuters)

In [17]:
#creating list of possible stopwords from nltk library
stop = stopwords.words('english')

def cleanText(txt):
  # lowercaing
    txt = txt.lower()
  # removing stopwords
    txt = ' '.join([word for word in txt.split() if word not in (stop)])
  # removing encoding error sym
    txt = re.sub('[^a-z$0-9]', ' ', txt.lower())
    return txt 

In [19]:
X['Text'] = X['Text'].apply(cleanText)

In [20]:
# configuration settingof LSTM (According to Research Paper)
voc_size = 121713 # no of unique tokens 
sent_length = 1000
Test_size = 0.3
batch_size = 120
EPOCHS = 30 
Learning_rate = 0.00001
embedding_vector_features = 100

In [23]:
onehot_repr = [one_hot(words, voc_size) for words in X['Text']]

In [24]:
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size=Test_size,shuffle=True, random_state=42)

In [31]:
model = Sequential()
# Add layers to the model
model.add(Input(shape=(sent_length,)))
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features))
model.add(LSTM(100, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid',))
# Compile the model
optimizer = SGD(learning_rate=Learning_rate,)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 1000, 100)           │      12,171,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,251,801 (46.74 MB)

 Trainable params: 12,251,801 (46.74 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=batch_size)

Epoch 1/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 1004s 4s/step - accuracy: 0.5175 - loss: 0.6930
Epoch 2/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 674s 3s/step - accuracy: 0.5187 - loss: 0.6929
Epoch 3/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 641s 2s/step - accuracy: 0.5179 - loss: 0.6929
Epoch 4/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 637s 2s/step - accuracy: 0.5161 - loss: 0.6929
Epoch 5/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 637s 2s/step - accuracy: 0.5187 - loss: 0.6929
Epoch 6/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 644s 2s/step - accuracy: 0.5225 - loss: 0.6928
Epoch 7/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 636s 2s/step - accuracy: 0.5157 - loss: 0.6929
Epoch 8/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.5203 - loss: 0.6928
Epoch 9/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 617s 2s/step - accuracy: 0.5273 - loss: 0.6928
Epoch 10/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 613s 2s/step - accuracy: 0.5239 - loss: 0.6928
Epoch 11/30
262/262 ━━━━━━━━━━━━━━━━━━━━ 609s 2s/step - accuracy: 0.5236 - loss: 0.6928
Epoch 12/30
262/262 ━━━━━━━━━━━━━━━━━━━━

In [35]:
y_pred = model.predict(X_test)

421/421 ━━━━━━━━━━━━━━━━━━━━ 96s 225ms/step


In [37]:
y_pred[:5]

array([[0.5033276 ],
       [0.50248235],
       [0.5030534 ],
       [0.49835432],
       [0.49690786]], dtype=float32)

In [39]:
print(classification_report(y_test,y_pred.round()))

              precision    recall  f1-score   support

           0       0.55      0.35      0.43      6474
           1       0.55      0.73      0.63      6996

    accuracy                           0.55     13470
   macro avg       0.55      0.54      0.53     13470
weighted avg       0.55      0.55      0.53     13470

